In [75]:
#importing the necessary libraries
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import time

In [76]:
#defining the function to fetch the game details
#batching the requests to avoid hitting the API rate limits
def fetch_game_details(game_ids, batch_size=10):
    games_data = []
    
    for i in range(0, len(game_ids), batch_size):
        batch_ids = game_ids[i:i+batch_size]
        response = requests.get(f"https://boardgamegeek.com/xmlapi2/thing?id={','.join(map(str, batch_ids))}&stats=1")
        if response.status_code != 200:
            raise Exception("API request failed")
        
        #parsing the XML response
        
        root = ET.fromstring(response.content)
        
        for item in root.findall('item'):
            game = {}
            game['name'] = item.find("name[@type='primary']").get('value')
            game['year_published'] = item.find("yearpublished").get('value')
            game['min_players'] = item.find("minplayers").get('value')
            game['max_players'] = item.find("maxplayers").get('value')
            game['min_playtime'] = item.find("minplaytime").get('value')
            game['max_playtime'] = item.find("maxplaytime").get('value')
            game['min_age'] = item.find("minage").get('value')  

            #fetching categories and mechanics
            game['categories'] = [link.get('value') for link in item.findall("link[@type='boardgamecategory']")]
            game['mechanics'] = [link.get('value') for link in item.findall("link[@type='boardgamemechanic']")]

            #fetch the rating counts, average ratings, and weight
            statistics = item.find("statistics/ratings")
            if statistics is not None:
                users_rated = statistics.find("usersrated")
                average_rating = statistics.find("average")
                average_weight = statistics.find("averageweight")
                if users_rated is not None and average_rating is not None and average_weight is not None:
                    game['users_rated'] = int(users_rated.get('value'))
                    game['average_rating'] = float(average_rating.get('value'))
                    game['weight'] = float(average_weight.get('value'))
                else:
                    game['users_rated'] = 0
                    game['average_rating'] = 0.0
                    game['weight'] = 0.0
            else:
                game['users_rated'] = 0
                game['average_rating'] = 0.0
                game['weight'] = 0.0

            games_data.append(game)
        
        #accounting for the API rate limits
        time.sleep(5)
    
    #converting the list of dictionaries to a DataFrame
    df = pd.DataFrame(games_data)
    return df

def read_game_ids_from_file(file_path):
    with open(file_path, 'r') as file:
        game_ids_str = file.read().strip()
        game_ids = [int(id.strip()) for id in game_ids_str.split(',')]
    return game_ids

#reading the game IDs from the file and fetching the game details
file_path = 'game_ids.txt' 
game_ids = read_game_ids_from_file(file_path)
game_details_df = fetch_game_details(game_ids)

#printing the DataFrame
print(game_details_df.head())

                name year_published min_players max_players min_playtime  \
0              CATAN           1995           3           4           60   
1        Carcassonne           2000           2           5           30   
2           Pandemic           2008           2           4           45   
3          7 Wonders           2010           2           7           30   
4  Terraforming Mars           2016           1           5          120   

  max_playtime min_age                                         categories  \
0          120      10                            [Economic, Negotiation]   
1           45       7                     [Medieval, Territory Building]   
2           45       8                                          [Medical]   
3           30      10  [Ancient, Card Game, City Building, Civilizati...   
4          120      12  [Economic, Environmental, Industry / Manufactu...   

                                           mechanics  users_rated  \
0  [Chainin

In [110]:
#exporting to csv
game_details_df.to_csv('game_details.csv', index=False)